In [26]:
from pandas_datareader import data, wb
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import datetime
%matplotlib inline

In [27]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [28]:
import chart_studio.plotly as py

In [29]:
import cufflinks as cf

In [30]:
init_notebook_mode(connected=True)

In [31]:
cf.go_offline()

In [281]:
stock = {}  # creating empty dict to store dataframes
stock_list = ['BKNG','EXPE','ALGT','LYV','SIX','FUN','DIS','PENN','DAL','UAL','AAL','LUV','ALK','AC','CCL','RCL','NCLH','LVS','MGM','WYNN','CZR','ERI','MAR','HLT','H','CHH','WH','PK','MTN','VAC','^BSESN','^NSEI']
# ^ creating list of all stock tickers

In [282]:
# creating dictionary 'd' of tickers and company name
d = {'ticker':['BKNG','EXPE','ALGT','LYV','SIX','FUN','DIS','PENN','DAL','UAL','AAL','LUV','ALK','AC','CCL','RCL','NCLH','LVS','MGM','WYNN','CZR','ERI','MAR','HLT','H','CHH','WH','PK','MTN','VAC','^BSESN','^NSEI'],
     'Name':['Booking Holdings','Expedia Group','Allegiant Travel','Live Nation','Six Flags','Cedar Fair','The Walt Disney Co','Penn National Gaming','Delta Air Lines','United Airlines','American Airlines','Southwest Airlines','Alaska Air Group','Air Canada','Carnival','Royal Caribbean Cruises','Norwegian Cruise Lines','Las Vegas Sands','MGM Resorts International','Wynn Resorts','Caesars Entertainment','Eldorado Resorts','Marriott International','Hilton','Hyatt Hotels','Choice Hotels International','Wyndham Hotels & Resorts','Park Hotels','Vail Resorts','Marriott Vacations Worldwide','S&P BSE SENSEX','NIFTY 50']}

In [283]:
# creating df from dictionary 'd'
comp_df = pd.DataFrame.from_dict(d)

In [284]:
# writing a function that will make a separate dataframe for each stock.
for df in stock_list:
    symbol = df                      
    data_source='yahoo'
    start_date = '2020-01-01'
    end_date = '2020-04-30'
    stock[df] = data.DataReader(symbol, data_source, start_date, end_date)  # storing the resulting df in dict
    stock[df] = stock[df][['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']] # repositioning columns for convenience
    stock[df].name = df

In [9]:
stock['BKNG'].shape

(84, 6)

In [10]:
stock['ALK'].shape

(84, 6)

In [11]:
stock['BKNG'].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 84 entries, 2019-12-31 to 2020-04-30
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       84 non-null     float64
 1   High       84 non-null     float64
 2   Low        84 non-null     float64
 3   Close      84 non-null     float64
 4   Adj Close  84 non-null     float64
 5   Volume     84 non-null     int64  
dtypes: float64(5), int64(1)
memory usage: 4.6 KB


In [12]:
stock['BKNG'].describe()

,Open,High,Low,Close,Adj Close,Volume
count,84.000000,84.000000,84.000000,84.000000,84.000000,8.400000e+01
mean,1668.030720,1700.370595,1636.982143,1666.452853,1666.452853,6.183095e+05
std,306.049163,289.653874,315.829782,304.538314,304.538314,2.646170e+05
min,1166.979980,1226.140015,1107.290039,1152.239990,1152.239990,1.985000e+05
25%,1378.979980,1426.425018,1352.837524,1375.292511,1375.292511,4.005000e+05
50%,1680.070007,1731.460022,1640.735046,1680.609985,1680.609985,5.797000e+05
75%,1961.250000,1975.940002,1941.442505,1961.415009,1961.415009,7.815250e+05
max,2090.250000,2094.000000,2076.939941,2086.899902,2086.899902,1.338000e+06


In [13]:
stock['BKNG'].head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-12-31,2046.510010,2057.989990,2039.969971,2053.729980,2053.729980,226900
2020-01-02,2068.399902,2077.409912,2053.219971,2074.580078,2074.580078,329000
2020-01-03,2042.469971,2067.689941,2035.000000,2065.479980,2065.479980,294000
2020-01-06,2050.000000,2059.790039,2035.000000,2047.400024,2047.400024,384000
2020-01-07,2047.390015,2072.830078,2038.540039,2068.050049,2068.050049,345300


In [211]:
# defining a function that calculates adjusted closing prices on 19/2, 24/3 and the percent decline between these 2 dates
def all_adj_c(x):
    tick = x.name
    adj_c_19 = x.loc['2020-02-19']['Adj Close']
    adj_c_23 = x.loc['2020-03-23']['Adj Close']
    adj_c_p = ((adj_c_19 - adj_c_23)/adj_c_19)*100
    return tick, adj_c_19, adj_c_23, adj_c_p

In [191]:
adj_c = all_adj_c(stock['BKNG'])

In [192]:
type(adj_c)

tuple

In [212]:
# defining a function that plots a line plot of the adjusted close price of the selected stocks.
# horizontal line at 23 March to show the severe decline in stock price.

def plot_f(*args):
    import plotly.graph_objects as go
    fig = go.Figure()
    for every_stock in args:
        fig.add_trace(go.Scatter(x=every_stock.index, y=every_stock['Adj Close'],mode='lines+markers',name=every_stock.name))
    fig.update_layout(shapes=[
    dict(
      type= 'line',
      yref= 'paper', y0= 0, y1= 1,
      xref= 'x', x0= '2020-03-23', x1= '2020-03-23'),])
    fig.show()

In [213]:
# defining a function that creates a dataframe of the adjusted close price of the selected stocks.
# it displays the Adj Close on 19 February, Adj Close on 23 March and Percent Decline in stock value.
def df_f(*args):
    temp_list=[]
    for every_stock in args:
        adj_c = all_adj_c(every_stock)
        temp_list.append(adj_c)
    temp_df = pd.DataFrame(temp_list, columns =['Ticker', 'Adj Close on 19 February', 'Adj Close on 23 March', 'Percent Decline'])
    tem_df_1 = temp_df.merge(comp_df,left_on='Ticker',right_on='ticker',how='left')
    return tem_df_1[['ticker','Name','Adj Close on 19 February', 'Adj Close on 23 March', 'Percent Decline']]


In [224]:
# Booking industry
plot_f(stock['BKNG'],stock['EXPE'],stock['ALGT'])
booking = df_f(stock['BKNG'],stock['EXPE'],stock['ALGT'])
booking

,ticker,Name,Adj Close on 19 February,Adj Close on 23 March,Percent Decline
0,BKNG,Booking Holdings,1968.489990,1152.239990,41.465794
1,EXPE,Expedia Group,121.428833,50.849998,58.123621
2,ALGT,Allegiant Travel,163.988739,71.709999,56.271388


In [225]:
# Entertainment industry
plot_f(stock['LYV'],stock['SIX'],stock['FUN'],stock['DIS'],stock['PENN'])
entertainment = df_f(stock['LYV'],stock['SIX'],stock['FUN'],stock['DIS'],stock['PENN'])
entertainment

,ticker,Name,Adj Close on 19 February,Adj Close on 23 March,Percent Decline
0,LYV,Live Nation,76.080002,37.330002,50.933227
1,SIX,Six Flags,37.641918,11.190000,70.272504
2,FUN,Cedar Fair,53.177879,18.440001,65.323926
3,DIS,The Walt Disney Co,141.300003,85.760002,39.306440
4,PENN,Penn National Gaming,37.099998,9.800000,73.584904


In [226]:
# Airlines industry
plot_f(stock['DAL'],stock['UAL'],stock['AAL'],stock['LUV'],stock['ALK'],stock['AC'])
airlines = df_f(stock['DAL'],stock['UAL'],stock['AAL'],stock['LUV'],stock['ALK'],stock['AC'])
airlines

,ticker,Name,Adj Close on 19 February,Adj Close on 23 March,Percent Decline
0,DAL,Delta Air Lines,58.509998,22.219999,62.023586
1,UAL,United Airlines,79.419998,26.250000,66.947871
2,AAL,American Airlines,28.330000,10.250000,63.819273
3,LUV,Southwest Airlines,56.891048,33.730000,40.711236
4,ALK,Alaska Air Group,65.169998,24.030001,63.127204
5,AC,Air Canada,43.676025,26.105900,40.228307


In [227]:
# Cruise & Casino industry
plot_f(stock['CCL'],stock['RCL'],stock['NCLH'],stock['LVS'],stock['MGM'],stock['WYNN'],stock['CZR'],stock['ERI'])
cruise = df_f(stock['CCL'],stock['RCL'],stock['NCLH'],stock['LVS'],stock['MGM'],stock['WYNN'],stock['CZR'],stock['ERI'])
cruise

,ticker,Name,Adj Close on 19 February,Adj Close on 23 March,Percent Decline
0,CCL,Carnival,42.840000,12.000000,71.988796
1,RCL,Royal Caribbean Cruises,109.918922,28.190001,74.353824
2,NCLH,Norwegian Cruise Lines,52.020000,9.670000,81.410996
3,LVS,Las Vegas Sands,67.748451,43.450001,35.865691
4,MGM,MGM Resorts International,31.849909,9.148845,71.275131
5,WYNN,Wynn Resorts,135.036850,57.570000,57.367193
6,CZR,Caesars Entertainment,14.610000,6.030000,58.726897
7,ERI,Eldorado Resorts,69.449997,10.440000,84.967603


In [228]:
# Hotels & Resorts industry
plot_f(stock['MAR'],stock['HLT'],stock['H'],stock['CHH'],stock['WH'],stock['PK'],stock['MTN'],stock['VAC'])
hotels = df_f(stock['MAR'],stock['HLT'],stock['H'],stock['CHH'],stock['WH'],stock['PK'],stock['MTN'],stock['VAC'])
hotels

,ticker,Name,Adj Close on 19 February,Adj Close on 23 March,Percent Decline
0,MAR,Marriott International,147.040421,70.129997,52.305633
1,HLT,Hilton,112.497368,63.330002,43.705348
2,H,Hyatt Hotels,89.730331,47.490002,47.074751
3,CHH,Choice Hotels International,107.832420,53.373211,50.503558
4,WH,Wyndham Hotels & Resorts,59.371952,24.984449,57.918767
5,PK,Park Hotels,21.756109,6.394948,70.606194
6,MTN,Vail Resorts,242.470413,140.040237,42.244402
7,VAC,Marriott Vacations Worldwide,125.817032,43.840000,65.155751


In [272]:
# combining all df to plot the total percent decline bar chart
all_comp = pd.concat([booking, entertainment, airlines, cruise, hotels])
all_comp_sorted = all_comp.sort_values(by='Percent Decline',ascending=True)

In [304]:
all_comp_sorted.head()

,ticker,Name,Adj Close on 19 February,Adj Close on 23 March,Percent Decline
3,LVS,Las Vegas Sands,67.748451,43.450001,35.865691
3,DIS,The Walt Disney Co,141.300003,85.760002,39.306440
5,AC,Air Canada,43.676025,26.105900,40.228307
3,LUV,Southwest Airlines,56.891048,33.730000,40.711236
0,BKNG,Booking Holdings,1968.489990,1152.239990,41.465794


In [274]:
# horizontal bar chart of percent decline of all companies
import plotly.express as px
fig = px.bar(all_comp_sorted, y='ticker', x='Percent Decline',color_continuous_scale=px.colors.sequential.ice_r, hover_data=['Name'], color='Percent Decline',height=900,orientation='h',labels={'ticker':'Companies'})
fig.show()

plotting sensex and nifty 50. added dropdown for better visualisation.

In [329]:
# plotting sensex and nifty 50
import plotly.graph_objects as go
fi = go.Figure()
fi.add_trace(go.Scatter(x=stock['^BSESN'].index, y=stock['^BSESN']['Adj Close'],mode='lines+markers',name=stock['^BSESN'].name))
fi.add_trace(go.Scatter(x=stock['^NSEI'].index, y=stock['^NSEI']['Adj Close'],mode='lines+markers',name=stock['^NSEI'].name))

# adding dropdown for better visualisation    
fi.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'Both',
                  method = 'update',
                  args = [{'visible': [True, True]},
                          {'title': 'Both',
                           'showlegend':True}]),
             dict(label = '^BSESN',
                  method = 'update',
                  args = [{'visible': [True, False]},
                          {'title': '^BSESN',
                           'showlegend':True}]),
             dict(label = '^NSEI',
                  method = 'update',
                  args = [{'visible': [False, True]},
                          {'title': '^NSEI',
                           'showlegend':True}]),
            ])
        )
    ])

fi.show()
india = df_f(stock['^BSESN'],stock['^NSEI'])
india

,ticker,Name,Adj Close on 19 February,Adj Close on 23 March,Percent Decline
0,^BSESN,S&P BSE SENSEX,41323.000000,25981.240234,37.126442
1,^NSEI,NIFTY 50,12125.900391,7610.250000,37.239712


In [349]:
for k,v in stock.items():
    print(v['Adj Close'].min())

1152.239990234375
45.650001525878906
68.81999969482422
29.5
10.359999656677246
13.449999809265137
85.76000213623047
4.519999980926514
21.350000381469727
21.280000686645508
9.390000343322754
29.110000610351562
23.559999465942383
26.105899810791016
7.96999979019165
22.329999923706055
7.769999980926514
37.68000030517578
7.139098644256592
43.02000045776367
3.5199999809265137
7.099999904632568
59.08000183105469
55.939998626708984
36.560001373291016
53.37321090698242
21.550708770751953
4.6681227684021
131.72999572753906
36.810001373291016
25981.240234375
7610.25
